# Import Keras and Tensorflow

In [0]:
import keras
print("Using Keras",keras.__version__)
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)

Using TensorFlow backend.


Using Keras 2.2.5
TensorFlow version: 2.2.0-rc2


# Read text-file with train/test sequences

In [0]:
#If you want to read a part of the data, edit the number of lines to read and uncomment 'videos_list = [next(f) for x in range(N)]'

#read train data
N = 100 #number of lines to read
path = "/content/gdrive/My Drive/Thesis/Code/Train.txt"
with open(path, 'r') as f:
    #read N lines with train sequences 
    #videos_list = [next(f) for x in range(N)]
    
    #read all lines with train sequences
    videos_list = f.readlines()

#read test data    
N = 20 #number of lines to read
path = "/content/gdrive/My Drive/Thesis/Code/Test.txt"
with open(path, 'r') as f:
    #read N lines with test sequences 
    #videos_list_test = [next(f) for x in range(N)]

    #read all lines with test sequences
    videos_list_test = f.readlines()

# Generate sequences of the images & save as numpy file

In [0]:
import cv2

#function to generate one batch from input sequence
def generate_video_batch(batch_filepath, batch_framestart):
  """
    Parameters:
        batch_filepath: this is the path to the frame from one batch
        batch_framestart: this is the frame start number of the batch in a list of frames from input video
    Output:
         Return one batch from input sequence in a numpy format
  """
  batch_images = []
  for frame in range(batch_framestart, batch_framestart + 16):
    read_img = cv2.imread((batch_filepath
                           + "/"
                           + "{:0>6d}".format(frame)
                           + ".jpg")) # path of image
    image = cv2.cvtColor(read_img, cv2.COLOR_BGR2RGB)
    
    image = cv2.resize(read_img, (112, 112)) #, interpolation=cv2.INTER_AREA)
    from google.colab.patches import cv2_imshow
    #cv2_imshow(image)
    #print(cv2.imshow('image',image))

    #print("De shape is: ", image.shape)
    batch_images.append(image / 255) 
  return batch_images

#function to generate batches of sequences
def generateData(videos_list):
    """
    Parameters:
        videos_list: this is a list which contains paths to the frames of the sequences
    Output:
         Return numpy file with all sequences 
  """
    x = []
    y = []
    count=1
    for batch in videos_list:
      read_batch = batch.split(" ")
      data_location = read_batch[0].split("/")[5:]
      batch_filepath = "/content/gdrive/My Drive/Colab Notebooks/" + str(data_location[0]) + "/" + str(data_location[1])
      #batch_filepath = read_batch[0]
      batch_framestart = int(read_batch[1])
      label = int(read_batch[2]) # one hot encoding
              
      batch_images = generate_video_batch(batch_filepath, batch_framestart)
      y.append(label)
      x.append(batch_images)
      print(count)
      count=count+1
    
    x = np.array(x)
    y = np.array(y)

    return x, y


In [ ]:
x_data_train, y_data_train = generateData(videos_list)
np.save('/content/gdrive/My Drive/Thesis/Code/C3D/X_data_OriginalImagesTrain', x_data_train)
np.save('/content/gdrive/My Drive/Thesis/Code/C3D/Y_data_OriginalImagesTrain', y_data_train)

In [ ]:
x_data_test, y_data_test = generateData(videos_list_test)
np.save('/content/gdrive/My Drive/Thesis/Code/C3D/X_data_OriginalImagesTest', x_data_test)
np.save('/content/gdrive/My Drive/Thesis/Code/C3D/Y_data_OriginalImagesTest', y_data_test)


In [0]:
#load the saved data
x_data_array = np.load('/content/gdrive/My Drive/Thesis/Code/C3D/X_data_OriginalImagesTrain.npy')
y_data_array = np.load('/content/gdrive/My Drive/Thesis/Code/C3D/Y_data_OriginalImagesTrain.npy')
x_data_test_array = np.load('/content/gdrive/My Drive/Thesis/Code/C3D/X_data_OriginalImagesTest.npy')
y_data_test_array = np.load('/content/gdrive/My Drive/Thesis/Code/C3D/Y_data_OriginalImagesTest.npy')

#get summary of loaded data
def data_summary(X_train, y_train, X_test, y_test):
    """Summarize current state of dataset"""
    print('Train images shape:', X_train.shape)
    print('Train labels shape:', y_train.shape)
    print('Test images shape:', X_test.shape)
    print('Test labels shape:', y_test.shape)
    print('Train labels:', y_train)
    print('Test labels:', y_test)

data_summary(x_data_array, y_data_array, x_data_test_array, y_data_test_array)

Train images shape: (1869, 16, 112, 112, 3)
Train labels shape: (1869,)
Test images shape: (625, 16, 112, 112, 3)
Test labels shape: (625,)
Train labels: [0 0 1 ... 1 1 1]
Test labels: [0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 1 0 1
 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 0 0 0 1
 0 1 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 1
 0 1 1 0 1 0 0 0 1 0 1 1 0 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 1 0 1
 1 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1
 1 1 1 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1
 0 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 1 0 0 1
 0 0 1 0 0 1 1 0 1 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 0
 1 1 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 1 1 0 1 1 0 0 0 1
 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 1 0 1
 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 0 